In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_excel("Data_Train.xlsx")
test = pd.read_excel("Data_Test.xlsx")
submission = pd.read_excel("Sample_submission.xlsx")
train["source"] = "train"
test["source"] = "test"
df = pd.concat([train,test])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16921 entries, 0 to 4230
Data columns (total 10 columns):
CITY             16774 non-null object
COST             12690 non-null float64
CUISINES         16921 non-null object
LOCALITY         16793 non-null object
RATING           16917 non-null object
RESTAURANT_ID    16921 non-null int64
TIME             16921 non-null object
TITLE            16921 non-null object
VOTES            15315 non-null object
source           16921 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 1.4+ MB


In [4]:
df = df.drop(["CUISINES",'CITY', 'LOCALITY', 'TIME', 'TITLE'],axis=1)

In [5]:
df.head()

,COST,RATING,RESTAURANT_ID,VOTES,source
0,1200.0,3.6,9438,49 votes,train
1,1500.0,4.2,13198,30 votes,train
2,800.0,3.8,10915,221 votes,train
3,800.0,4.1,6346,24 votes,train
4,300.0,3.8,15387,165 votes,train


In [6]:
df['VOTES'] = df['VOTES'].astype(str)
df['VOTES'] = df.VOTES.apply(lambda x : x[0:-6])

In [7]:
df["VOTES"] = pd.to_numeric(df["VOTES"], errors='coerce')

In [8]:
df['VOTES'] = df['VOTES'].astype('Int64')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16921 entries, 0 to 4230
Data columns (total 5 columns):
COST             12690 non-null float64
RATING           16917 non-null object
RESTAURANT_ID    16921 non-null int64
VOTES            15315 non-null Int64
source           16921 non-null object
dtypes: Int64(1), float64(1), int64(1), object(2)
memory usage: 809.7+ KB


In [10]:
df = df.drop(['RESTAURANT_ID'],axis=1)

In [16]:
df.isna().sum()

COST      4231
RATING       0
VOTES        0
source       0
dtype: int64

In [12]:
df["RATING"] = pd.to_numeric(df["RATING"], errors='coerce')
df["RATING"]=df.RATING.round(0)

In [13]:
df["RATING"]= df['RATING'].astype('Int64')

In [14]:
df["RATING"]=df.RATING.fillna(0)

In [15]:
df["VOTES"]=df.VOTES.transform(lambda x : x.fillna(x.median()))

In [17]:
train_final = df[df.source=="train"]
test_final = df[df.source=="test"]

In [18]:
train_final = train_final.drop(["source"],axis=1)
train_final.shape

(12690, 3)

In [19]:
test_final = test_final.drop(["source"],axis=1)
test_final.shape

(4231, 3)

In [28]:
test_final = test_final.drop(columns=["COST"])
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4231 entries, 0 to 4230
Data columns (total 2 columns):
RATING    4231 non-null Int64
VOTES     4231 non-null Int64
dtypes: Int64(2)
memory usage: 107.4 KB


In [22]:
train_X = train_final.drop(columns=["COST"])

In [23]:
train_Y = train_final["COST"]

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [25]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12690 entries, 0 to 12689
Data columns (total 2 columns):
RATING    12690 non-null Int64
VOTES     12690 non-null Int64
dtypes: Int64(2)
memory usage: 322.2 KB


In [26]:
model.fit(train_X, train_Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [29]:
dtrain_predictions = model.predict(train_X)

In [30]:
from sklearn import metrics
#Print model report:
print("\nModel Report")
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(train_Y.values, dtrain_predictions)))
    
#Predict on testing data:
test_final["COST"] =  model.predict(test_final)


Model Report
RMSE : 599.5


In [37]:
Linear_submission = test_final[["COST"]]
Linear_submission.to_excel("D:\Participants_Data_Final\Participants_Data_Final\LinearRegression1.xlsx", encoding='utf8',index=False)